# Predicting the Shooting Percentage of Basketball Stars

> **Introduction:** This is the Question 1 of APOAI 2025 Mock Competition, and it is also the first question of the NOAI 2024 (China).

## I. Question Overview

A CSV file of basketball stars' shooting data is given, which is stored in the training set `data_train.csv`. The meaningful variables are named as follows:

- `loc_x`: The horizontal position of the basketball star on the court when shooting, which has been normalized. There is no need to consider the unit.
- `loc_y`: The vertical position of the basketball star on the court when shooting, which has been normalized. There is no need to consider the unit.
- `minutes_remaining`: The number of minutes remaining in the current quarter, which has been normalized. There is no need to consider the unit.
- `shot_distance`: The distance from the shooting position to the basket, which has been normalized. There is no need to consider the unit.
- `shot_made_flag`: Whether the shot is made or not. If it is made, the value is 1; otherwise, the value is 0.
- `shot_id`: The unique identifier of the sample.

Now, it is hoped that you can build a model based on the historical data of the basketball star to predict the shooting result of the star. There are 20,000 data points in the training set, and the test set has approximately 5,000 data points. The format is similar to that of the training set, and the contestants cannot access it during the competition.

## II. Data Set

Address of the training set: [Training Set](https://bohrium.dp.tech/competitions/5135119121?tab=datasets).

## III. Task

Please use PyTorch to implement a multi-layer perceptron (neural network) model to predict whether a contestant can make a shot at different positions. The specific requirements are as follows:

1. The input consists of 2 features (`loc_x`, `loc_y`), and the output is 1 label (`shot_made_flag`). 1 indicates that the shot is made, and 0 indicates that the shot is missed.
2. Only linear layers and activation functions after linear layers can be used, and at most 3 linear layers can be used. Please build the neural network directly and do not use `nn.Sequential()` nesting. The scoring system cannot detect the network structure inside `nn.Sequential()`, and a score of 0 will be directly given.
3. The activation function can only be selected from `nn.Linear`, `nn.ReLU`, `nn.Sigmoid`, `nn.Tanh`, `nn.ELU`, `nn.LeakyReLU`, `nn.PreLU`. Each linear layer can have at most 8 neurons.
4. The loss function, optimizer (solver), and learning rate can be freely selected.

## IV. Submission

Please submit a compressed file named `submission.zip`. After decompression, it should contain the model file `submission_model.py` and the model parameter file `submission_dic.pth`. The specific requirements are as follows:

1. Save the class definition of the model and the required precursor libraries in `submission_model.py`.
2. Save the trained model parameters in `submission_dic.pth`. The model parameters will be loaded during scoring.
3. You can refer to the method in `baseline.ipynb` to generate the `submission.zip` file on the platform for submission. You can also download the data set to the local machine, train the model, and then package it into a `submission.zip` file for submission.

> **Address of `baseline.ipynb`:** [Question 1 of APOAI2025 Mock Competition_baseline](https://bohrium.dp.tech/notebooks/18211783382).

## V. Scoring

1. When the number of linear layers and the number of neurons in the neural network meet the requirements, the score is the accuracy rate of the test set.
2. When the number of linear layers and the number of neurons in the neural network do not meet the requirements, the score is 0.

> **Remarks:** The leaderboard A uses 50% of the data in the test set, which can be displayed in real time during the competition to help contestants debug the model. The leaderboard B uses the remaining 50% of the data in the test set and is calculated after the competition ends. The score of the leaderboard B is the final score.

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import zipfile
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

seed = 42
os.environ["PYTHONHASHSEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Load data

In [2]:
df = pd.read_csv("data_train.csv")
df

,loc_x,loc_y,minutes_remaining,shot_distance,shot_made_flag,shot_id
0,0,0,7,0,1,1
1,-3,130,9,13,0,2
2,82,8,7,8,0,3
3,100,101,10,14,1,4
4,0,0,9,0,1,5
...,...,...,...,...,...,...
24995,97,43,1,10,0,24996
24996,18,77,2,7,0,24997
24997,234,4,0,23,0,24998
24998,75,20,2,7,0,24999


In [3]:
df.isna().any()

loc_x                False
loc_y                False
minutes_remaining    False
shot_distance        False
shot_made_flag       False
shot_id              False
dtype: bool

In [4]:
df.describe()

,loc_x,loc_y,minutes_remaining,shot_distance,shot_made_flag,shot_id
count,25000.000000,25000.000000,25000.000000,25000.000000,25000.00000,25000.000000
mean,7.125400,91.287880,4.885120,13.458520,0.44744,12500.500000
std,110.029921,88.289321,3.452593,9.397722,0.49724,7217.022701
min,-250.000000,-44.000000,0.000000,0.000000,0.00000,1.000000
25%,-67.250000,4.000000,2.000000,5.000000,0.00000,6250.750000
50%,0.000000,72.000000,5.000000,15.000000,0.00000,12500.500000
75%,94.000000,160.000000,8.000000,21.000000,1.00000,18750.250000
max,248.000000,791.000000,11.000000,79.000000,1.00000,25000.000000


In [5]:
input_cols = ["loc_x", "loc_y"]  # Specified in the task statement
output_col = "shot_made_flag"

In [6]:
X = df[input_cols]
y = df[output_col]
X.shape, y.shape

((25000, 2), (25000,))

In [7]:
X_tensor = torch.tensor(X.to_numpy(), dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, random_state=42)
X_train.shape, y_train.shape

(torch.Size([18750, 2]), torch.Size([18750]))

## Define model and train

In [9]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.act = nn.Tanh()
        self.fc1 = nn.Linear(2, 8)
        self.fc2 = nn.Linear(8, 8)
        self.fc3 = nn.Linear(8, 1)
        
    def forward(self, x):
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
def train(model, optimizer, criterion, X, y, num_epochs):
    train_losses = []
    for epoch in range(num_epochs):
        model.train()

        outputs = model(X).squeeze()
        loss = criterion(outputs, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")
    
    return train_losses

In [11]:
@torch.no_grad()
def test(model, X, y):
    model.eval()

    probs = torch.sigmoid(model(X).squeeze())
    preds = (probs >= 0.5).to(torch.uint8)

    return accuracy_score(preds, y)

In [12]:
model = MyModel()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [13]:
train(model, optimizer, criterion, X_train, y_train, 15);

Epoch [1/15], Loss: 0.699416995048523
Epoch [2/15], Loss: 0.698686420917511
Epoch [3/15], Loss: 0.6979755163192749
Epoch [4/15], Loss: 0.6972848773002625
Epoch [5/15], Loss: 0.6966170072555542
Epoch [6/15], Loss: 0.6959756016731262
Epoch [7/15], Loss: 0.6953611373901367
Epoch [8/15], Loss: 0.6947712302207947
Epoch [9/15], Loss: 0.6942040920257568
Epoch [10/15], Loss: 0.6936602592468262
Epoch [11/15], Loss: 0.6931406855583191
Epoch [12/15], Loss: 0.6926440000534058
Epoch [13/15], Loss: 0.692168653011322
Epoch [14/15], Loss: 0.6917139291763306
Epoch [15/15], Loss: 0.691280722618103


In [14]:
test(model, X_test, y_test)

0.52288

## Save for submission

In [15]:
py_filename = "submission_model.py"
pth_filename = "submission_dic.pth"
zip_filename = "submission.zip"  # Will submit this zip to the grader

In [16]:
torch.save(model.state_dict(), pth_filename)

In [17]:
model_code = """  
import torch
import torch.nn as nn


class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.act = nn.Tanh()
        self.fc1 = nn.Linear(2, 8)
        self.fc2 = nn.Linear(8, 8)
        self.fc3 = nn.Linear(8, 1)
        
    def forward(self, x):
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.fc3(x)
        return x
""".lstrip()

with open(py_filename, "w") as f:
    f.write(model_code)

In [18]:
with zipfile.ZipFile(zip_filename, "w") as zipf:
    for file in [py_filename, pth_filename]:
        zipf.write(file, os.path.basename(file))

## Score

Leaderboard A accuracy: 0.5604

Leaderboard B accuracy: 0.5677